In [2]:
import requests
import urllib.request
import json

In [11]:
# 연습
header = {"Authorization": "KakaoAK "+ "{MY_REST_API_KEY}"}
url = "https://dapi.kakao.com/v3/search/book"

Text = urllib.parse.quote(text)
nPage = 1
# 쿼리스트링에는 검색어, 페이지 번호, 검색옵션, 한페이지 당 검색량 등이 들어간다.
queryString = "?query="+Text+"&page="+str(nPage)+"&sort=latest"+"&size=50"

#json파일로 주기 때문에 json.loads를 사용한다.
r = requests.get(url+queryString, headers=header)
print(json.loads(r.text))

{'documents': [{'authors': ['데이비드 화이트'], 'contents': '언어는 우리와 함께하며 세상에 대해 가르쳐 준다. 생각과 사고를 확장시키며, 자신 밖의 세상에 대해 이야기해 준다. 이들은 말이 많지 않지만 이들이 전하는 한 마디 한 마디가 폐부를 꿰뚫는다. 이번에 로만 출판사에서 새롭게 출간된 《위로》는 언어가 우리에게 건네는 말을 담은 책이다. 이 책은 숨겨진 단어의 의미 속에서 우리에게 건네는 위안과 격려를 찾아 준다. 곁에 계속 함께했음에도 그동안 깨닫지 못했던 따스함을 말이다.  위안은 극도로 힘들고 추한 순간에', 'datetime': '2021-07-17T00:00:00.000+09:00', 'isbn': '1197301615 9791197301612', 'price': 16000, 'publisher': '로만', 'sale_price': 14400, 'status': '정상판매', 'thumbnail': 'https://search1.kakaocdn.net/thumb/R120x174.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flbook%2Fimage%2F5747565%3Ftimestamp%3D20210629143836', 'title': '위로(양장본 HardCover)', 'translators': ['이상원'], 'url': 'https://search.daum.net/search?w=bookpage&bookId=5747565&q=%EC%9C%84%EB%A1%9C%28%EC%96%91%EC%9E%A5%EB%B3%B8+HardCover%29'}, {'authors': ['키티 오메라'], 'contents': '대유행하면서 격리와 봉쇄로 모두가 사회적 거리두기를 해야 했던 2020년 3월, 키티 오메라가 시 한 편을 지어 SNS에 게재한다. 이 시는 희망의 불씨가 되어 전 세계적으로 빠르게 퍼져 나갔다. 시는 지치고 힘들었던 사람들의 마음을 위로해 주고 치유해 주었다. 더불어 감동과

In [12]:
# 함수화하여 데이터베이스를 만들기 쉽게 한다.
header = {"Authorization": "KakaoAK "+ "{MY_REST_API_KEY}"}
url = "https://dapi.kakao.com/v3/search/book"
def searchBook(text, nPage):
    Text = urllib.parse.quote(text)
    queryString = "?query="+Text+"&page="+str(nPage)+"&sort=latest"+"&size=50"

    r = requests.get(url+queryString, headers=header)
    return json.loads(r.text)

In [49]:
import re
# 카테고리를 미리 정해서 카테고리별 도서 목록을 구축해보자
categoryList = ["수학","과학","영어","정치","지식","요리","주식","위로"]

resultList = []

# 문제집은 최대한 제외하기 위해 불용어를 지정한다.
stop_words = ["기출", "시험", "문제", "수능", "학습서", '고등', '강사']

for category in categoryList:
    cnt = 0 #50개의 도서만 가져온다.
    resultDict = {}
    bookList = []
    resultDict['category'] = category
    for i in range(1,6):
        if cnt > 50:
            break
        jsonData = searchBook(category, i)
        document = jsonData['documents']
        for script in document:
            if cnt > 50:
                break
            bookDict = {}
            isAppend = True
            for word in stop_words:
                #불용어 제거
                if script['contents'].find(word) != -1:
                    isAppend = False
                    break;
            if isAppend and script['contents'] != "":
                bookDict['title'] = script['title']
                bookDict['authors'] = script['authors']
                bookDict['date'] = re.search('\d{4}-\d{2}-\d{2}', script['datetime']).group() #시간정보를 제외한 날짜정보만 가져온다.
                bookDict['contents'] = script['contents']
                bookList.append(bookDict)
                cnt += 1
    resultDict['bookList'] = bookList
    resultList.append(resultDict)

#print(resultList)

In [50]:
# bookData.json에서 구축한 데이터를 볼 수 있다.
file_path = 'data/bookData.json'
with open(file_path, 'w', encoding = 'utf-8') as outfile:
    outfile.write(json.dumps(resultList, ensure_ascii=False))